In [68]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
import matplotlib.pyplot as plt
from statsmodels.stats.multitest import multipletests

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [41]:
genes = pd.read_csv('gene_high_throughput_sequencing.csv')

In [42]:
genes.Diagnosis.unique()

array(['normal', 'early neoplasia', 'cancer'], dtype=object)

In [50]:
genes.iloc[::10,:6]

,Patient_id,Diagnosis,LOC643837,LOC100130417,SAMD11,NOC2L
0,STT5425_Breast_001_normal,normal,1.257614,2.408148,13.368622,9.494779
10,STT5502_Breast_010_normal,normal,3.558222,6.270213,18.380624,11.162901
20,STT5756_Breast_022_normal,normal,2.234576,2.936809,13.791320,22.254160
30,STT5440_Breast_003_EN,early neoplasia,3.386331,12.379176,28.350470,14.547256
40,STT5612_Breast_017_EN,early neoplasia,4.567425,6.078425,14.829839,14.625044
50,STT5450_Breast_006_DCIS,cancer,1.609022,10.632542,28.344988,18.462903
60,STT5581_Breast_016_IDC,cancer,0.903290,1.729668,15.775280,18.386628
70,STT5763_Breast_022_IDC,cancer,3.733734,8.860505,32.538666,21.585069


In [49]:
#Расчет экспрессии

def  fold_change(C, T):
    if T > C:
        return T / C
    return -C / T


In [61]:
genes_expres = pd.DataFrame(columns=['gene', 'normal_to_early_tt', 'early_to_cancer_tt'])

In [62]:

for i, gene in enumerate(genes.drop(['Patient_id', 'Diagnosis'], axis=1).columns.values.tolist()):
    genes_expres.loc[i, 'gene'] = gene
    genes_expres.loc[i, 'normal_to_early_tt'] = sc.stats.ttest_ind(genes.loc[genes.Diagnosis == 'normal', gene],
                                                                   genes.loc[genes.Diagnosis == 'early neoplasia', gene],
                                                                   equal_var=False)[1]
    genes_expres.loc[i, 'early_to_cancer_tt'] = sc.stats.ttest_ind(genes.loc[genes.Diagnosis == 'early neoplasia', gene],\
                                                                   genes.loc[genes.Diagnosis == 'cancer', gene],
                                                                   equal_var=False)[1]


In [65]:
first_case = genes_expres['normal_to_early_tt'] < 0.05
second_case = genes_expres['early_to_cancer_tt'] < 0.05

print(genes_expres.loc[first_case].shape[0])
print(genes_expres.loc[second_case].shape[0])

1575
3490


In [63]:
genes_expres.head()

,gene,normal_to_early_tt,early_to_cancer_tt
0,LOC643837,0.690766,0.413735
1,LOC100130417,3.17853e-05,0.653429
2,SAMD11,0.0602727,0.0795556
3,NOC2L,0.826429,0.287581
4,KLHL17,0.0498762,0.463292


In [77]:
reject, p_corrected, a1, a2 = multipletests(genes_expres['early_to_cancer_tt'], 
                                            alpha = 0.025, 
                                            method = 'holm')
genes_expres['early_to_cancer_tt_holm'] = p_corrected
genes_expres['early_to_cancer_tt_holm_rej'] = reject

In [78]:
reject, p_corrected, a1, a2 = multipletests(genes_expres['normal_to_early_tt'], 
                                            alpha = 0.025, 
                                            method = 'holm')
genes_expres['normal_to_early_tt_holm'] = p_corrected
genes_expres['normal_to_early_tt_holm_rej'] = reject

In [82]:
print(genes_expres[genes_expres['normal_to_early_tt_holm_rej']].shape[0])
print(genes_expres[genes_expres['early_to_cancer_tt_holm_rej']].shape[0])

2
79


In [83]:
genes_expres.head()

,gene,normal_to_early_tt,early_to_cancer_tt,normal_to_early_tt_holm,normal_to_early_tt_holm_rej,early_to_cancer_tt_holm,early_to_cancer_tt_holm_rej
0,LOC643837,0.690766,0.413735,1,False,1,False
1,LOC100130417,3.17853e-05,0.653429,0.500174,False,1,False
2,SAMD11,0.0602727,0.0795556,1,False,1,False
3,NOC2L,0.826429,0.287581,1,False,1,False
4,KLHL17,0.0498762,0.463292,1,False,1,False


In [98]:
for gene in genes_expres['gene']:
    t = genes.loc[genes.Diagnosis == 'early neoplasia', gene]
    c = genes.loc[genes.Diagnosis == 'normal', gene]
    genes_expres.loc[genes_expres['gene'] == gene, 'fold_change_normal_to_early'] = fold_change(np.mean(c), np.mean(t))
    
    t = genes.loc[genes.Diagnosis == 'cancer', gene]
    c = genes.loc[genes.Diagnosis == 'early neoplasia', gene]
    genes_expres.loc[genes_expres['gene'] == gene, 'fold_change_early_to_cancer'] = fold_change(np.mean(c), np.mean(t))

In [87]:
genes_expres[genes_expres['normal_to_early_tt_holm_rej']]

,gene,normal_to_early_tt,early_to_cancer_tt,normal_to_early_tt_holm,normal_to_early_tt_holm_rej,early_to_cancer_tt_holm,early_to_cancer_tt_holm_rej,fold_change_normal_to_early
7244,PCSK4,7.95543e-07,0.0939552,0.0125274,True,1,False,1.509785
9820,EEF1A2,8.49874e-08,0.574408,0.00133838,True,1,False,1.974868


In [88]:
'''
for gene in genes_expres.loc[genes_expres['early_to_cancer_tt_holm_rej'], 'gene']:
    t = genes.loc[genes.Diagnosis == 'cancer', gene]
    c = genes.loc[genes.Diagnosis == 'early neoplasia', gene]
    genes_expres.loc[genes_expres['gene'] == gene, 'fold_change_early_to_cancer'] = fold_change(np.mean(c), np.mean(t))
    
'''    

In [99]:
genes_expres[genes_expres['fold_change_early_to_cancer'] > 1.5]

,gene,normal_to_early_tt,early_to_cancer_tt,normal_to_early_tt_holm,normal_to_early_tt_holm_rej,early_to_cancer_tt_holm,early_to_cancer_tt_holm_rej,fold_change_normal_to_early,fold_change_early_to_cancer,early_to_cancer_tt_bh,early_to_cancer_tt_bh_rej,normal_to_early_tt_bh,normal_to_early_tt_bh_rej
8,ISG15,0.240148,0.00074024,1,False,1,False,1.066094,2.458867,0.0175562,True,0.794398,False
47,GABRD,0.00301703,2.27604e-07,1,False,0.00357521,True,1.311855,1.843860,8.61452e-05,True,0.160231,False
125,C1orf127,0.909869,0.0145512,1,False,1,False,1.013395,1.511742,0.108493,False,0.990361,False
182,MFAP2,0.884923,0.000888547,1,False,1,False,1.018841,1.903690,0.0194886,True,0.988348,False
186,PADI3,0.0643574,0.344465,1,False,1,False,1.568106,1.503323,0.615784,False,0.550287,False
190,ACTL8,0.347963,0.102324,1,False,1,False,1.168570,1.560690,0.330612,False,0.867712,False
193,PAX7,0.281326,0.0413797,1,False,1,False,1.223940,2.146681,0.203138,False,0.828048,False
212,PLA2G2D,0.538042,0.0507605,1,False,1,False,-1.118322,1.527802,0.227483,False,0.934747,False
234,EPHA8,0.00762016,0.201019,1,False,1,False,1.871956,1.766175,0.470946,False,0.235761,False
236,C1QC,0.232637,0.00626105,1,False,1,False,-1.172679,1.523307,0.0656012,False,0.788914,False


In [105]:
reject, p_corrected, a1, a2 = multipletests(genes_expres['early_to_cancer_tt'], 
                                            alpha = 0.025, 
                                            method = 'fdr_bh')
genes_expres['early_to_cancer_tt_bh'] = p_corrected
genes_expres['early_to_cancer_tt_bh_rej'] = reject

reject, p_corrected, a1, a2 = multipletests(genes_expres['normal_to_early_tt'], 
                                            alpha = 0.025, 
                                            method = 'fdr_bh')
genes_expres['normal_to_early_tt_bh'] = p_corrected
genes_expres['normal_to_early_tt_bh_rej'] = reject

In [106]:
print(genes_expres[genes_expres['normal_to_early_tt_bh_rej']].shape[0])
print(genes_expres[genes_expres['early_to_cancer_tt_bh_rej']].shape[0])

4
832


In [108]:
first = genes_expres['fold_change_normal_to_early'] > 1.5
second = genes_expres['fold_change_normal_to_early'] < -1.5
third = genes_expres['normal_to_early_tt_bh_rej']

genes_expres[third & (first | second)].shape[0]

4

In [109]:
first = genes_expres['fold_change_early_to_cancer'] > 1.5
second = genes_expres['fold_change_early_to_cancer'] < -1.5
third = genes_expres['early_to_cancer_tt_bh_rej']

genes_expres[third & (first | second)].shape[0]

524